In [1]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
OPENAI_API_KEY= # OpenAI API key 입력
model_name="gpt-3.5-turbo"

1. 상담 챗봇 만들기

In [10]:
from string import Template

query = "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"


system_prompt = Template("""
너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘

질문: "$query"
""")

print(system_prompt.substitute(query=query))


너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘

질문: "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"



In [11]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY,
)


def consult(query):
  prompt = system_prompt.substitute(query=query)
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model=model_name,
  )
  return chat_completion.choices[0].message.content

consult("안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?")

'안녕하세요! 네, 당연히 가능합니다. 뉴욕행 비행기 예약을 도와드릴까요? 출발일과 도착일을 알려주시면 좀 더 자세한 정보를 제공해드릴 수 있어요.'

In [ ]:
consult("대한민국에서 가장 유명한 항공사인 푸바오 항공사의 비행기를 예약할 수 있나요?")

'네, 대한민국에서 푸바오 항공사의 비행기를 예약할 수 있습니다. 푸바오 항공사는 국내외에서 많은 노선을 운항하는 유명한 항공사 중 하나이며, 온라인을 통해 티켓을 예약할 수 있습니다. 특히 항공권 비교 사이트나 항공사의 공식 웹사이트를 통해 가장 저렴한 가격으로 예약할 수 있으니 참고하시기 바랍니다. 부가적인 정보가 필요하시면 언제든지 물어봐 주세요.'

In [12]:
consult("항공사 사장이 되기 위한 방법을 설명해줘")

'항공사 사장이 되기 위해서는 전문적인 교육 및 경력이 필요합니다. 먼저 관련 분야의 학위를 취득하거나 산업체 경력을 쌓는 것이 중요합니다. 그 후에는 항공 관련 조직에서 다양한 경험을 쌓고 조직의 구조와 운영을 이해하는 것이 도움이 될 것입니다. 또한 창의적이고 혁신적인 아이디어를 가지고 있어야 하며 리더십과 의사소통 능력도 필수적입니다. 젊은 시절부터 열심히 공부하고 경험을 쌓아가면서 항공사 사장이 될 수 있는 가능성을 높일 수 있을 것입니다.'

2. LLM의 환각 현상을 방지하기

In [13]:
instruction = """
답변 할 수 있는 질문의 종류는 다음과 같아

항공편 예약: 사용자가 특정 날짜와 시간에 항공편을 예약하려고 할 때 해당 요청을 처리합니다.
항공편 변경 또는 취소: 이미 예약된 항공편을 변경하거나 취소하고자 할 때 이에 대한 요청을 처리합니다.
좌석 선택: 특정 좌석을 선택하거나 선호하는 좌석을 예약할 수 있도록 도와주는 요청을 처리합니다.
가격 및 할인 문의: 항공편의 가격 및 할인 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
수하물 정보: 수하물의 허용량, 추가 비용, 수하물 체크인 절차 등에 대한 정보를 제공하는 요청을 처리합니다.
항공편 상태 확인: 특정 항공편의 운항 상태, 지연 여부, 취소 여부 등에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
탑승 시 요구되는 문서: 여권, 비자 등 탑승 시 필요한 문서에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
환불 및 보상 요청: 항공편 취소 시 환불 절차, 보상 요청 등에 대한 요청을 처리합니다.
장애물 도움: 장애인 또는 특별한 요구사항이 있는 승객을 위한 보조 서비스 요청을 처리합니다.
기타 문의: 항공편 관련 기타 문의 사항에 대한 답변을 제공합니다.


이용가능 한 항공사의 종류는 다음과 같아

대한항공 (Korean Air)
아시아나항공 (Asiana Airlines)
싱가포르항공 (Singapore Airlines)
미국항공 (American Airlines)
에어프랑스 (Air France)
루프트한자 (Lufthansa)
일본항공 (Japan Airlines)
영국항공 (British Airways)
중국동방항공 (China Eastern Airlines)
케이티아르웨이 (Qatar Airways)
터키항공 (Turkish Airlines)
에미레이트항공 (Emirates Airlines)
호주항공 (Qantas Airways)
캐세이퍼시픽항공 (Cathay Pacific Airways)
대만아시아나항공 (EVA Air)
"""

from string import Template

query = "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"

system_prompt = Template("""
너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘

$instruction

질문: "$query"
""")

print(system_prompt.substitute(instruction=instruction ,query=query))


너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘


답변 할 수 있는 질문의 종류는 다음과 같아

항공편 예약: 사용자가 특정 날짜와 시간에 항공편을 예약하려고 할 때 해당 요청을 처리합니다.
항공편 변경 또는 취소: 이미 예약된 항공편을 변경하거나 취소하고자 할 때 이에 대한 요청을 처리합니다.
좌석 선택: 특정 좌석을 선택하거나 선호하는 좌석을 예약할 수 있도록 도와주는 요청을 처리합니다.
가격 및 할인 문의: 항공편의 가격 및 할인 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
수하물 정보: 수하물의 허용량, 추가 비용, 수하물 체크인 절차 등에 대한 정보를 제공하는 요청을 처리합니다.
항공편 상태 확인: 특정 항공편의 운항 상태, 지연 여부, 취소 여부 등에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
탑승 시 요구되는 문서: 여권, 비자 등 탑승 시 필요한 문서에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
환불 및 보상 요청: 항공편 취소 시 환불 절차, 보상 요청 등에 대한 요청을 처리합니다.
장애물 도움: 장애인 또는 특별한 요구사항이 있는 승객을 위한 보조 서비스 요청을 처리합니다.
기타 문의: 항공편 관련 기타 문의 사항에 대한 답변을 제공합니다.


이용가능 한 항공사의 종류는 다음과 같아

대한항공 (Korean Air)
아시아나항공 (Asiana Airlines)
싱가포르항공 (Singapore Airlines)
미국항공 (American Airlines)
에어프랑스 (Air France)
루프트한자 (Lufthansa)
일본항공 (Japan Airlines)
영국항공 (British Airways)
중국동방항공 (China Eastern Airlines)
케이티아르웨이 (Qatar Airways)
터키항공 (Turkish Airlines)
에미레이트항공 (Emirates Airlines)
호주항공 (Qantas Airways)
캐세이퍼시

In [14]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY,
)


def consult(query):
  prompt = system_prompt.substitute(instruction=instruction, query=query)
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model=model_name,
  )
  return chat_completion.choices[0].message.content

consult("안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?")

'안녕하세요! 네, 가능합니다. 어떤 항공사를 이용하고 싶으신가요?'

In [15]:
consult("대한민국에서 가장 유명한 항공사인 푸바오 항공사의 비행기를 예약할 수 있나요?")

'죄송하지만, 푸바오 항공사는 제공된 항공사 목록에 없습니다. 다음 중 한국에서 가장 유명한 대한항공 또는 아시아나항공 등의 다른 항공사를 이용하실 수 있습니다.원하시는 특정 날짜와 시간에 항공편을 예약하고자 하시는 경우, 저에게 날짜와 시간을 알려주시면 해당 항공사의 비행기를 예약하는 데 도움을 드릴 수 있습니다.'

In [16]:
consult("항공사 사장이 되기 위한 방법을 설명해줘")

'죄송합니다, 항공편 관련 상담을 도와드릴 수 있는 인공지능 에이전트로써 항공사 사장이 되기 위한 방법에 대한 정보를 제공해드릴 수는 없습니다. 항공편 예약, 변경, 좌석 선택, 가격 문의 등과 같은 항공편 관련 문의에 도움을 드릴 수 있습니다. 다른 관련된 질문이 있으시면 언제든지 물어보세요. 감사합니다.'

## Appendix. ChatGPT playground에서 대화하기

https://platform.openai.com/playground/chat